In [2]:
import requests
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
import time
import datetime

In [3]:
def get_track_features(id):
    meta = sp.track(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    spotify_url = meta['external_urls']['spotify']
    album_cover = meta['album']['images'][0]['url']
    track_info = [name, album, artist, spotify_url, album_cover]
    return track_info

def get_track_ids(time_frame):
    track_ids = []
    for song in time_frame['items']:
        track_ids.append(song['id'])
    return track_ids


def get_artist_features(id):
    meta = sp.track(id)
    artist_id = meta['album']['artists'][0]['uri']
    artist = sp.artist(artist_id)
    url = artist['external_urls']['spotify']
    artist_genre = artist['genres']
    image = artist['images'][0]['url']
    name = artist['name']
    artist_info = [name, artist_genre, image, url]
    return artist_info


In [4]:
client_id = 'client_id'
client_secret = 'client_secret'
redirect_uri = 'http://127.0.0.1:8080'
scope = "user-top-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, 
                                               client_secret=client_secret, 
                                               redirect_uri=redirect_uri, 
                                               scope=scope))

top_tracks_short = sp.current_user_top_tracks(limit=50, offset=0, time_range="short_term")

In [5]:
#TOP‌ 5 MUSICS OF USER

track_ids = get_track_ids(top_tracks_short)
# loop over track ids 
tracks = []
for i in range(len(track_ids)):
    time.sleep(.5)
    track = get_track_features(track_ids[i])
    tracks.append(track)

df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'spotify_url', 'album_cover'])
df.head(5)

,name,album,artist,spotify_url,album_cover
0,Avalanche,That's The Spirit,Bring Me The Horizon,https://open.spotify.com/track/4EyPadLFhtWojU7...,https://i.scdn.co/image/ab67616d0000b273736145...
1,Astronaut In The Ocean,Astronaut In The Ocean,Our Last Night,https://open.spotify.com/track/2qjH6wZfR0g11Zi...,https://i.scdn.co/image/ab67616d0000b273274c95...
2,Retrospection,Macro,Jinjer,https://open.spotify.com/track/01W4AJOJGOWxlN3...,https://i.scdn.co/image/ab67616d0000b2734a6f9e...
3,Cyberpunk 2077 Theme Song (Metal),Cyberpunk 2077 Theme Song (Metal),Artificial Fear,https://open.spotify.com/track/0fyvnfgqC425qIF...,https://i.scdn.co/image/ab67616d0000b273778100...
4,Death Is No Dread Enemy,Every Nothing,Tides Of Man,https://open.spotify.com/track/1aYxXhXvscuchfr...,https://i.scdn.co/image/ab67616d0000b273ae64b3...


In [6]:
# TOP‌ 5 MUSICS OF ARTIST

artists =[]
for i in track_ids:
    time.sleep(.5)
    artist = get_artist_features(i)
    artists.append(artist)
    
artists_df = pd.DataFrame(artists, columns =['name', 'artist_genre', 'image', 'url'])
artists_df['count'] = artists_df.groupby('name')['name'].cumcount()
artists_df['count'] = artists_df['count'] + 1
top_artists = artists_df.sort_values(by='count',ascending=False)
top_artists = top_artists.drop(columns='artist_genre')
top_artists.drop_duplicates(subset='name',keep='first',inplace = True)
top_artists.head(5)

,name,image,url,count
21,Bring Me The Horizon,https://i.scdn.co/image/ab6761610000e5eb2321fd...,https://open.spotify.com/artist/1Ffb6ejR6Fe5Ia...,3
31,Pink Floyd,https://i.scdn.co/image/e69f71e2be4b67b82af90f...,https://open.spotify.com/artist/0k17h0D3J5Vfsd...,3
30,Conquer Divide,https://i.scdn.co/image/ab6761610000e5eb017323...,https://open.spotify.com/artist/4ZznAOb4aE3va5...,3
19,The Pretty Reckless,https://i.scdn.co/image/ab6761610000e5eb78d174...,https://open.spotify.com/artist/2R57sY41L9XvGP...,2
34,Jinjer,https://i.scdn.co/image/ab6761610000e5eb3a579f...,https://open.spotify.com/artist/7o6cOczXTB8ioT...,2


In [7]:
#TOP 5 GENRES

from collections import Counter

genres = []
for i in artists_df['artist_genre']:
    genres = genres + i

genres_dict = {i:genres.count(i) for i in genres}
top_genres = dict(Counter(genres_dict).most_common(5))
top_genres_chart = pd.DataFrame.from_dict(top_genres, orient='index',columns=['count'])
top_genres_chart

,count
metalcore,12
rock,11
post-screamo,7
pixie,6
art rock,6


In [8]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

img = Image.open('templates/musics.jpg')
draw = ImageDraw.Draw(img)
# font = ImageFont.truetype(<font-file>, <font-size>)
font = ImageFont.truetype('font/Gotham.ttf', 45)

#Writing Top five artists
gg = 1044
for i in top_artists.head(5)['name']:
    if len(i)>16:
        i = i.replace(i[16:],'...')
    draw.text((85, gg),i,(255,69,48),font=font)
    gg = gg + 53
gg = 1044

#Writing top 5 musics
for i in df.head(5)['name']:
    if len(i)>16:
        i = i.replace(i[16:],'...')
    draw.text((630, gg),i,(255,69,48),font=font)
    gg = gg + 53

#Writing top genre
top_genre = list(top_genres.keys())[0]
font2 = ImageFont.truetype('font/Gotham.ttf', 55)
draw.text((590, 1510),top_genre.capitalize(),(255,69,48),font=font2)


#Insert Top artist Image
import urllib.request
urllib.request.urlretrieve(list(top_artists.head(5)["image"])[0],"file_name")
artist_image= Image.open("file_name")
artist_image = artist_image.resize((800,800))
img.paste(artist_image,(140, 67))

#Add my Info here
font3 = ImageFont.truetype('font/Gotham.ttf', 40)
draw.text((700, 1850),"Spotify Wrapped\nBy Hossein M",(0,0,0),font=font3)

#saving the image
img.save('Spotify-Wrapped.jpg')

